In [3]:
import pickle
pathdata = "../data/interim/"
filename = pathdata + "Zlvl_Vabs"
with open(filename, 'rb') as f:
    Vabsbyz_sort = pickle.load(f)

## TODO:  MAKE box plot of velocity as function of station (look seaborn functions)